In [2]:
import requests
import re
import json
from tqdm import tqdm

url = "https://mcgqlapi.com/api"

albums_file = open('albums.txt', 'r')
albums_lines = albums_file.readlines()

albums = {}

for album_line in albums_lines:
    elements = [e.strip() for e in album_line.split(" - ")]
    try:
        artist, genre = elements[1].split("(")[0].strip(), elements[1].split("(")[1].replace(")", "").strip()
        album = {
            "title": elements[0],
            "artist": artist,
            "genre": genre,
            "year": int(elements[2]),
            "label": elements[3],
            "sales": elements[4]
        }

        albums[elements[0]] = {"data": {"album": album}}

    except IndexError:
        continue

with open('albums.json', 'w') as output_file:
    output_file.write(json.dumps(albums))

In [3]:
import json
import pandas as pd
import csv


with open('albums.json', 'r') as file:
    data = json.load(file)

def convert_sales(sales):
    sales = sales.replace(',', '')
    if 'million' in sales:
        sales = float (sales.replace('million', ''))
        sales = int(sales * 1000000)
    else:
        sales = int(sales)
    return sales

with open('albums.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Type', 'Artist', 'Genre', 'ReleaseYear', 'Label/Studio', 'CriticScore'])
    for album_title, album_data in data.items():
        album = album_data['data']
        sales = convert_sales(album['album']['sales'])
        if sales >= 40000000:
            score = "Excellent"
        elif sales >= 10000000:
            score = "Good"
        elif sales >= 1000000:
            score = "Mediocre"
        else:
            score = "Bad"
        if((album.get('album'))):
            writer.writerow([
                album['album']['title'],
                "Album",
                album['album']['artist'],
                album['album']['genre'],
                album['album']['year'],
                album['album']['label'],
                score
            ])

In [39]:
pd.read_csv("albums.csv")

,Title,Type,Artist,Genre,ReleaseYear,Label/Studio,CriticScore
0,Thriller,Album,Michael Jackson,Pop/R&B,1982,Epic Records,Excellent
1,The Dark Side of the Moon,Album,Pink Floyd,Progressive Rock,1973,Harvest Records,Excellent
2,Back in Black,Album,AC/DC,Hard Rock,1980,Atlantic Records,Excellent
3,The Bodyguard Soundtrack,Album,Various Artists,Soundtrack/Pop,1992,Arista Records,Excellent
4,Bat Out of Hell,Album,Meat Loaf,Rock,1977,Epic Records,Excellent
...,...,...,...,...,...,...,...
96,You Forgot It in People,Album,Broken Social Scene,Indie Rock,2002,Arts & Crafts,Bad
97,Let It Come Down,Album,Spiritualized,Space Rock,2001,Arista Records,Bad
98,The Argument,Album,Fugazi,Post-Hardcore,2001,Dischord Records,Bad
99,Person Pitch,Album,Panda Bear,Experimental Pop,2007,Paw Tracks,Bad
